In [ ]:
!java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)


In [ ]:
# Import the required Module
!pip install -q tabula-py

In [ ]:
import tabula
from tabula.io import read_pdf
import pandas as pd
import requests
import time
from io import BytesIO
from PyPDF2 import PdfReader

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pdfreader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Create an empty dataframe to store the text from each PDF
df = pd.DataFrame(columns=['Roll Number', 'Text'])

# Iterate through the roll numbers and create URLs
for i in range(1, 450):
    url = f'https://clerk.house.gov/legislative/proxy-letters/117/1/votes/2021/roll{i}.pdf'

    # Download the PDF file from the URL
    response = requests.get(url)
    pdf_file = BytesIO(response.content)

    # Use PyPDF2 to extract text from the PDF
    pdf_reader = PdfReader(pdf_file)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Split the text into a list of strings using newline characters as delimiters
    text_list = text.split('\n')

    # Create a new dataframe with one row for each string in the text_list
    temp_df = pd.DataFrame({'Roll Number': [i] * len(text_list), 'Text': text_list})

    # Append the new dataframe to the main dataframe
    df = pd.concat([df, temp_df], ignore_index=True)

# Save the dataframe to a CSV file
# df.to_csv('pdf_texts.csv', index=False)

In [ ]:
df.head()

In [ ]:
#The PDF structure makes it impossible for the words to appear in the proper places. 
#To clean them off, we get rid of words in improper places one after the other
#Starting with 'Members Recorded Pursuant to House Resolution 8, 117th Congress', which appear undder the text column.
df['Text'] = df['Text'].str.replace('Members Recorded Pursuant to House Resolution 8, 117th Congress', '')

In [ ]:
#We create a mask with rows where the line text starts with Roll Call No.
mask = df['Text'].str.startswith('Roll Call No.')

In [ ]:
#We create a filter with those that do not, because those are the rows we need.
df_filtered = df[~mask]

In [ ]:
df1 = df_filtered

In [ ]:

df2 = df1.dropna()
df2.head(10)

In [ ]:
#Create a date column using all rows that are dates.
df2['Date'] = df2['Text'].str.extract(r'(\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b)')


In [ ]:
# Apparently, because date only appear on each page, once we created a date column, not all lines will have a date in front of it
# But we need date in front of each line
# So we fill missing values with the preceding non-missing value
df2['Date'] = df2['Date'].fillna(method='ffill')

In [ ]:
#Now we delete all dates in the Text column
df2['Text'] = df2['Text'].str.replace(r'(\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b)', '')

In [ ]:
#More cleaning
# Delete rows where there is no value under Text
df3 = df2[df2['Text'].str.match(r'^\s*$') == False]

In [ ]:
#Clean up the name column
# Create a column with the first names before the bracket in the text column
df3['Voter'] = df3['Text'].str.split().str[0]

In [ ]:
df3.to_csv('HouseProxyVotes_1ndSession_117thCongress.csv')

In [ ]:
# Create an empty dataframe to store the text from each PDF
df4 = pd.DataFrame(columns=['Roll Number', 'Text'])

# Iterate through the roll numbers and create URLs
for i in range(1, 550):
    url = f'https://clerk.house.gov/legislative/proxy-letters/117/2/votes/2022/roll{i}.pdf'

    # Download the PDF file from the URL
    response = requests.get(url)
    pdf_file = BytesIO(response.content)

    # Use PyPDF2 to extract text from the PDF
    pdf_reader = PdfReader(pdf_file)
    text = ''
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Split the text into a list of strings using newline characters as delimiters
    text_list = text.split('\n')

    # Create a new dataframe with one row for each string in the text_list
    temp_df2 = pd.DataFrame({'Roll Number': [i] * len(text_list), 'Text': text_list})

    # Append the new dataframe to the main dataframe
    df4 = pd.concat([df4, temp_df2], ignore_index=True)

# Save the dataframe to a CSV file
# df.to_csv('pdf_texts.csv', index=False)

In [ ]:
df4.head()

,Roll Number,Text
0,1,Members Recorded Pursuant to House Resolution ...
1,1,Roll Call No. 1
2,1,"January 10, 2022"
3,1,
4,1,Adams (Brown (MD))


In [ ]:
df4['Text'] = df4['Text'].str.replace('Members Recorded Pursuant to House Resolution 8, 117th Congress', '')

In [ ]:
mask2 = df4['Text'].str.startswith('Roll Call No.')

In [ ]:
df_filtered2 = df4[~mask]

In [ ]:
df5 = df_filtered2

In [ ]:
df6 = df5.dropna()
df6.head(10)

,Roll Number,Text
0,1,
2,1,"January 10, 2022"
3,1,
4,1,Adams (Brown (MD))
5,1,Aguilar (Correa)
6,1,Auchincloss (Clark (MA))
7,1,Babin (Session)
8,1,Barragan (Beyer)
9,1,Bass (Cicilline)
10,1,Bera (Kildee)


In [ ]:
df6['Date'] = df6['Text'].str.extract(r'(\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b)')


In [ ]:
# fill missing values with last preceding non-missing value
df6['Date'] = df6['Date'].fillna(method='ffill')

In [ ]:
df2['Text'] = df2['Text'].str.replace(r'(\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b)', '')

<ipython-input-49-1763b643cfff>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Text'] = df2['Text'].str.replace(r'(\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}\b)', '')


In [ ]:
#Delete rows where there is no value under Text
df7 = df6[df6['Text'].str.match(r'^\s*$') == False]

In [ ]:
#Create a column with the first names before the bracket in the text column
df7['Voter'] = df7['Text'].str.split().str[0]

<ipython-input-54-483c07e45005>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Voter'] = df3['Text'].str.split().str[0]


In [ ]:
df7.to_csv('HouseProxyVotes_2ndSession_117thCongress.csv')